In [1]:
import os
import pickle
import time
import random

import torch

import numpy as np

from konlpy.tag import Twitter

from tqdm import tqdm

from seq2seq.dataset import Dataset
from seq2seq.model import Seq2seqModel
from seq2seq.trainer import Trainer, Evaluator

In [2]:
USE_CUDA = torch.cuda.is_available()
GPU_ID = 2

USE_CUDA

True

In [3]:
torch.backends.cudnn.enabled=False

In [4]:
# 물음표 느낌표 온점 쉼표 처리하도록 추가

# 1. Preprocess Dataset

In [5]:
SRC_FILE_PATH = 'dataset/korean_dialog/hangul_src.txt'
TGT_FILE_PATH = 'dataset/korean_dialog/hangul_tgt.txt'

EXPR_PATH = 'experiment_special_char_smaller_batch/'

DATASET_PKL = EXPR_PATH + 'dataset.pkl'
MODEL_PKL = EXPR_PATH + 'test_model10.model'

BATCH_SIZE = 256
MAX_LENGTH = 30

In [6]:
tagger = Twitter()

In [7]:
if not os.path.exists(EXPR_PATH):
    os.makedirs(EXPR_PATH)

if os.path.isfile(DATASET_PKL):
    print('Load dataset pickle')
    with open(DATASET_PKL, 'rb') as fp:
        dataset = pickle.load(fp)
    print('Source vocab : ', dataset.src_vocab.n_words)
    print('Target vocab : ', dataset.tgt_vocab.n_words)
else:
#    start_time = time.time()
    dataset = Dataset(
        src_file_path=SRC_FILE_PATH,
        tgt_file_path=TGT_FILE_PATH,
        max_length=MAX_LENGTH
    )
    dataset.prepareData(tagger=tagger)
#    print("--- %s seconds ---" % (time.time() - start_time))
    with open(DATASET_PKL, 'wb') as fp:
        pickle.dump(dataset, fp)

Load dataset pickle
Source vocab :  13745
Target vocab :  13682


# 2. Define Model

In [8]:
INPUT_SIZE = dataset.src_vocab_size
HIDDEN_SIZE = 1024
OUTPUT_SIZE = dataset.tgt_vocab_size

In [9]:
model = Seq2seqModel(
    name='test_model',
    input_size=INPUT_SIZE,
    hidden_size=HIDDEN_SIZE,
    output_size=OUTPUT_SIZE,
    max_length=MAX_LENGTH,
    bidirectional=True,
    gpu_id=GPU_ID
)

# 3. Train Model

In [10]:
trainer = Trainer(
    model=model,
    dataset=dataset,
    gpu_id=GPU_ID,
    print_interval=1,
    plot_interval=5,
    checkpoint_interval=5,
    expr_path=EXPR_PATH
)

In [11]:
#trainer.train(num_epoch=200, batch_size=BATCH_SIZE)

# 4. Test Model

In [12]:
evaluator = Evaluator(dataset, model)
evaluator.loadModel(EXPR_PATH+'test_model100.model')

In [13]:
evaluator.generateResponse('언제 여행 가시는건가요', beam_size=-1)

>  언제 여행 가시 는 건가 요
<  <SOS> 감사하겠 . <EOS>



In [14]:
evaluator.generateResponse('언제 여행 가시는건가요', beam_size=1)

>  언제 여행 가시 는 건가 요
<  <SOS> 감사하겠 . <EOS>



In [17]:
evaluator.evalModel(num=20, beam_size=5)

>  그럼 술 을 어디 에서 구입 할 수 있어 요 ?
<  <SOS> 네 , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ,

>  네 . 140 cm 이상 부터 이용할 수 있 습니다 .
<  <SOS> 가깝 시간 군요 군요 군요 군요 군요 군요 군요 군요 군요 군요 군요 군요 군요 군요 군요 군요 군요 군요 군요 군요 군요 군요 군요 군요 군요 군요 군요 군요 군요

>  안녕하세 요 ?
<  <SOS> 해보세 요 . <EOS>

>  그렇 다면 고기 가 안 들어간 다른 먹거리 는 없 습니까 ?
<  <SOS> 죄송합 다면 죄송합 다면 죄송합 다면 죄송합 죄송합 다면 죄송합 다면 죄송합 다면 죄송합 다면 죄송합 다면 죄송합 다면 죄송합 다면 죄송합 다면 죄송합 다면 죄송합 다면 죄송합 다면 죄송합 다면

>  네 , 그럼 다음 칸 에 다녀올 테 니 찾아보세 요 .
<  <SOS> . <EOS>

>  저 도 곧 내리는 걸 요 . 괜찮 으니 어서 앉으 세요 .
<  <SOS> 군요 요 군요 요 . 요 . 가진 요 . 연락처 알려 나면 요 요 . <EOS>

>  여기 서 북쪽 으로 출발하는 버스 는 뭐 든 괜찮 아요 .
<  <SOS> 생 생 처럼 낫 생 처럼 쯤 부탁드릴 않네 않네 않네 않네 않네 않네 않네 않네 않네 않네 않네 않네 않네 않네 않네 않네 않네 않네 않네 않네 않네 않네 않네

>  아뇨 . 인텔리 젠 시아 라고 하던 데요 .
<  <SOS> 커브 직원 이 직원 이 나중 캡 이 직원 이 취미 제품 이 캡 제품 이 제품 이 제품 이 제품 이 제품 이 제품 이 제품 이 제품 이 있

>  그 전 까지 관람 할 수 있 나요 ? 지금 이 4 시 인데 .
<  <SOS> 네 , 변경 들 일찍 일찍 일찍 일찍 일찍 일찍 일찍 일찍 일찍 일찍 일찍 일찍 일찍 일찍 일찍 일찍 일찍 일찍 일찍 일찍 일찍 일찍 일찍 일찍 일찍 일찍 일찍

>  1 킬로그램 에 150 원 입니 다 .
<  <SOS> 수수료